In [25]:
import os
from Bio.PDB import PDBParser, PDBIO, Select, Atom

# Directory containing PDB files
pdb_dir = "/home/hp/nayanika/github/GPX6/structures/"
output_dir = "/home/hp/nayanika/github/GPX6/structures/"
os.makedirs(output_dir, exist_ok=True)

# List of PDB files
pdb_files = ["original_mousecys.pdb"]

# List of mutations (residue_number, new_residue)
mutations = {
    47: "ALA",  # Mutate residue at position 47 to Alanine
    48: "TYR",  # Mutate residue at position 48 to Tyrosine
}

class MutateSelect(Select):
    def __init__(self, mutations):
        self.mutations = mutations

    def accept_residue(self, residue):
        res_id = residue.get_id()[1]
        if res_id in self.mutations:
            new_residue = self.mutations[res_id]
            residue.resname = new_residue
            
            # If mutating to ALA, remove OG if it exists
            if new_residue == "ALA":
                self.remove_og(residue)
            # If mutating to TYR, add the OH atom
            elif new_residue == "TYR":
                self.add_tyr_oh(residue)
                
        return True

    def remove_og(self, residue):
        """Remove the OG atom from ALA if it exists."""
        if "OG" in [atom.get_name() for atom in residue]:
            residue.detach_child("OG")
            print(f"Removed OG atom from ALA at residue {residue.get_id()[1]}")

    def add_tyr_oh(self, residue):
        """Manually add OH atom to TYR residue if it's missing."""
        atom_names = [atom.get_name() for atom in residue]
        if "OH" not in atom_names:
            cz = residue['CZ']
            # Calculate OH coordinates (slight offset along z-axis)
            oh_coords = [cz.coord[0], cz.coord[1], cz.coord[2] + 1.0]  # Adjust height for OH
            oh_atom = Atom.Atom(
                "OH", oh_coords, 1.0, 1.0, " ", "OH", residue.get_id()[1], element="O"
            )
            residue.add(oh_atom)
            print(f"Added OH atom to TYR at residue {residue.get_id()[1]} with coords {oh_coords}")

# Function to mutate residues in a PDB file
def mutate_pdb(pdb_file, output_file, mutations):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file)
    
    io = PDBIO()
    io.set_structure(structure)
    
    # Save the mutated structure
    io.save(output_file, MutateSelect(mutations))
    print(f"Mutated PDB saved to {output_file}")

# Apply mutations to all PDB files
for pdb_file in pdb_files:
    pdb_path = os.path.join(pdb_dir, pdb_file)
    output_path = os.path.join(output_dir, "S47A_F48Y.pdb")
    mutate_pdb(pdb_path, output_path, mutations)

print("All mutations completed successfully.")
